# Bost-dtn Analysis

In [16]:
import os
import json
import numpy as np
from pathlib import Path

rootdir = "/Users/eashan22/Desktop/Internship 2021/bbrv2/Brian's Project/bost-dtn"
# rootdir = "/home/eadhikarla/Desktop/Brian's Project/bost-dtn"

def traverse1(path):
    fileList = []
    c = 0
    for root, directories, files in os.walk(path):
        for file in files:
            if file.endswith("json"):
                fileList.append(os.path.join(root,file))
                c+=1
    print(f"total files: {c}")
    return fileList

paths = [("2021-06-09:14:35/pscheduler_bbr2_p1", "2021-06-09:14:35/pscheduler_cubic_p1", "2021-06-09:14:35/pscheduler_both_p16"),
         ("2021-06-15:02:12/pscheduler_bbr2_p1", "2021-06-15:02:12/pscheduler_cubic_p1", "2021-06-15:02:12/pscheduler_both_p16"),
         ("2021-06-16:02:12/pscheduler_bbr2_p1", "2021-06-16:02:12/pscheduler_cubic_p1", "2021-06-16:02:12/pscheduler_both_p16")
        ]

for p in paths:
    print(f"--- {p} ---")
    pscheduler_bbr2  = os.path.join(rootdir, p[0])
    filenames = traverse1(pscheduler_bbr2)

    data_seg = []
    tput_bbr2_p1 = []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            data = [json.loads(line) for line in open(f, 'r')]
            
            bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in enumerate(data):
                if "interval" in d.keys() and j==0:
                    start_time = data[j]['interval']['time'] # Start time of the test
                elif key2 in data[j].keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size

                    bbr2_data_seg = data[j][key2]
                    data_seg.append( bbr2_data_seg )

                    throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)/1e9
                    tput_bbr2_p1.append( throughput )
        
        except Exception as e:
            print(e)

    print("Throughput (P1)")
    print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p1):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p1):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p1)/np.mean(tput_bbr2_p1)):.5f}  |  Variance: {np.var(tput_bbr2_p1):.5f}")
    print("Data Segment (P1)")
    print(f"BBRv2 - Mean: {np.mean(data_seg):.5f}  |  Std. Dev.: {np.std(data_seg):.5f}  |  Coef. of Variance: {(np.std(data_seg)/np.mean(data_seg)):.5f}  |  Variance: {np.var(data_seg):.5f}")
    print("")

    pscheduler_cubic = os.path.join(rootdir, p[1])
    filenames = traverse1(pscheduler_cubic)

    data_seg = []
    tput_p1_cubic = []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            # The MongoDB JSON dump has one object per line, so this works for me.
            data = [json.loads(line) for line in open(f, 'r')]
            
            cubic_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in enumerate(data):
                if "interval" in d.keys() and j==0:
                    start_time = data[j]['interval']['time'] # Start time of the test
                elif key1 in data[j].keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size

                    cubic_data_seg = data[j][key1]
                    data_seg.append( cubic_data_seg )

                    throughput = (cubic_data_seg*mss*8)/(end_time-start_time)/1e9
                    tput_p1_cubic.append( throughput )
        
        except Exception as e:
            print(e)

    print("Throughput (P1)")
    print(f"CUBIC - Mean: {np.mean(tput_p1_cubic):.5f}  |  Std. Dev.: {np.std(tput_p1_cubic):.5f}  |  Coef. of Variance: {(np.std(tput_p1_cubic)/np.mean(tput_p1_cubic)):.5f}  |  Variance: {np.var(tput_p1_cubic):.5f}")
    print("Data Segment (P1)")
    print(f"CUBIC - Mean: {np.mean(data_seg):.5f}  |  Std. Dev.: {np.std(data_seg):.5f}  |  Coef. of Variance: {(np.std(data_seg)/np.mean(data_seg)):.5f}  |  Variance: {np.var(data_seg):.5f}")
    print("")

    pscheduler_both = os.path.join(rootdir, p[2])
    filenames = traverse1(pscheduler_both)

    data_seg_sum_cubic, data_seg_sum_bbr2 = [], []
    tput_cubic_p16, tput_bbr2_p16 = [], []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            data = [json.loads(line) for line in open(f, 'r')]
            
            throughput_cubic, throughput_bbr2, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in zip(range(len(data)),data):
                try:
                    if "interval" in d.keys() and j==0:
                        start_time = d['interval']['time'] # Start time of the test
                    if "streams" in d.keys():
                        end_time = data[j-1]['interval']['time'] # End time of the test
                        mss = data[j-1]['interval']['mss'] # maximum segment size
                        
                        cubic_data_seg_list, bbr2_data_seg_list = [], []
                        for j in range(len(d['streams'])):
                            if "cubic" in d['streams'][j]['cc']:
                                cubic_data_seg_list.append(d['streams'][j]['data_segs'])
                            elif "bbr2" in d['streams'][j]['cc']:
                                bbr2_data_seg_list.append(d['streams'][j]['data_segs'])
                        
                        data_seg_sum_cubic.append( sum(cubic_data_seg_list) )
                        throughput_cubic = (sum(cubic_data_seg_list)*mss*8)/(end_time-start_time)/1e9
                        tput_cubic_p16.append( throughput_cubic )

                        data_seg_sum_bbr2.append( sum(bbr2_data_seg_list) )
                        throughput_bbr2 = (sum(bbr2_data_seg_list)*mss*8)/(end_time-start_time)/1e9
                        tput_bbr2_p16.append( throughput_bbr2 )

                except Exception as e:
                    print(e)

        except Exception as e:
            print(e)

    print("Throughput (P16)")
    print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p16):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p16):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p16)/np.mean(tput_bbr2_p16)):.5f}  |  Variance: {np.var(tput_bbr2_p16):.5f}")
    print(f"CUBIC - Mean: {np.mean(tput_cubic_p16):.5f}  |  Std. Dev.: {np.std(tput_cubic_p16):.5f}  |  Coef. of Variance: {(np.std(tput_cubic_p16)/np.mean(tput_cubic_p16)):.5f}  |  Variance: {np.var(tput_cubic_p16):.5f}")

    print("Data Segment (P16)")
    print(f"BBRv2 - Mean: {np.mean(data_seg_sum_bbr2):.5f}  |  Std. Dev.: {np.std(data_seg_sum_bbr2):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_bbr2)/np.mean(data_seg_sum_bbr2)):.5f}  |  Variance: {np.var(data_seg_sum_bbr2):.5f}")
    print(f"CUBIC - Mean: {np.mean(data_seg_sum_cubic):.5f}  |  Std. Dev.: {np.std(data_seg_sum_cubic):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_cubic)/np.mean(data_seg_sum_cubic)):.5f}  |  Variance: {np.var(data_seg_sum_cubic):.5f}")
    print(50*"=")

--- ('2021-06-09:14:35/pscheduler_bbr2_p1', '2021-06-09:14:35/pscheduler_cubic_p1', '2021-06-09:14:35/pscheduler_both_p16') ---
total files: 105
Throughput (P1)
BBRv2 - Mean: 6.01623  |  Std. Dev.: 2.64542  |  Coef. of Variance: 0.43971  |  Variance: 6.99823
Data Segment (P1)
BBRv2 - Mean: 10593937.79048  |  Std. Dev.: 7304442.62257  |  Coef. of Variance: 0.68949  |  Variance: 53354882026455.64844

total files: 105
Throughput (P1)
CUBIC - Mean: 6.52317  |  Std. Dev.: 2.94479  |  Coef. of Variance: 0.45144  |  Variance: 8.67180
Data Segment (P1)
CUBIC - Mean: 11281322.24762  |  Std. Dev.: 7735380.38158  |  Coef. of Variance: 0.68568  |  Variance: 59836109647663.10156

total files: 109
Throughput (P16)
BBRv2 - Mean: 3.42120  |  Std. Dev.: 1.29872  |  Coef. of Variance: 0.37961  |  Variance: 1.68668
CUBIC - Mean: 4.30749  |  Std. Dev.: 2.00634  |  Coef. of Variance: 0.46578  |  Variance: 4.02542
Data Segment (P16)
BBRv2 - Mean: 16160038.23853  |  Std. Dev.: 25749337.16830  |  Coef. of Var

## ESnet Hosts

In [17]:
import os
import json
import numpy as np
from pathlib import Path

rootdir = "/Users/eashan22/Desktop/Internship 2021/bbrv2/Brian's Project/bost-dtn"
# rootdir = "/home/eadhikarla/Desktop/Brian's Project/bost-dtn"

def traverse2(path):
    fileList = []
    c = 0
    for root, directories, files in os.walk(path):
        for file in files:
            if file.endswith("json") and file.find("es.net")!=-1: # esnet host
                fileList.append(os.path.join(root,file))
                c+=1
    print(f"total files: {c}")
    return fileList

paths = [("2021-06-09:14:35/pscheduler_bbr2_p1", "2021-06-09:14:35/pscheduler_cubic_p1", "2021-06-09:14:35/pscheduler_both_p16"),
         ("2021-06-15:02:12/pscheduler_bbr2_p1", "2021-06-15:02:12/pscheduler_cubic_p1", "2021-06-15:02:12/pscheduler_both_p16"),
         ("2021-06-16:02:12/pscheduler_bbr2_p1", "2021-06-16:02:12/pscheduler_cubic_p1", "2021-06-16:02:12/pscheduler_both_p16")
        ]

for p in paths:
    print(f"--- {p} ---")
    pscheduler_bbr2  = os.path.join(rootdir, p[0])
    filenames = traverse2(pscheduler_bbr2)

    data_seg = []
    tput_bbr2_p1 = []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            data = [json.loads(line) for line in open(f, 'r')]
            
            bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in enumerate(data):
                if "interval" in d.keys() and j==0:
                    start_time = data[j]['interval']['time'] # Start time of the test
                elif key2 in data[j].keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size

                    bbr2_data_seg = data[j][key2]
                    data_seg.append( bbr2_data_seg )

                    throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)/1e9
                    tput_bbr2_p1.append( throughput )
        
        except Exception as e:
            print(e)

    print("Throughput (P1)")
    print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p1):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p1):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p1)/np.mean(tput_bbr2_p1)):.5f}  |  Variance: {np.var(tput_bbr2_p1):.5f}")
    print("Data Segment (P1)")
    print(f"BBRv2 - Mean: {np.mean(data_seg):.5f}  |  Std. Dev.: {np.std(data_seg):.5f}  |  Coef. of Variance: {(np.std(data_seg)/np.mean(data_seg)):.5f}  |  Variance: {np.var(data_seg):.5f}")
    print("")

    pscheduler_cubic = os.path.join(rootdir, p[1])
    filenames = traverse2(pscheduler_cubic)

    data_seg = []
    tput_p1_cubic = []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            # The MongoDB JSON dump has one object per line, so this works for me.
            data = [json.loads(line) for line in open(f, 'r')]
            
            cubic_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in enumerate(data):
                if "interval" in d.keys() and j==0:
                    start_time = data[j]['interval']['time'] # Start time of the test
                elif key1 in data[j].keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size

                    cubic_data_seg = data[j][key1]
                    data_seg.append( cubic_data_seg )

                    throughput = (cubic_data_seg*mss*8)/(end_time-start_time)/1e9
                    tput_p1_cubic.append( throughput )
        
        except Exception as e:
            print(e)

    print("Throughput (P1)")
    print(f"CUBIC - Mean: {np.mean(tput_p1_cubic):.5f}  |  Std. Dev.: {np.std(tput_p1_cubic):.5f}  |  Coef. of Variance: {(np.std(tput_p1_cubic)/np.mean(tput_p1_cubic)):.5f}  |  Variance: {np.var(tput_p1_cubic):.5f}")
    print("Data Segment (P1)")
    print(f"CUBIC - Mean: {np.mean(data_seg):.5f}  |  Std. Dev.: {np.std(data_seg):.5f}  |  Coef. of Variance: {(np.std(data_seg)/np.mean(data_seg)):.5f}  |  Variance: {np.var(data_seg):.5f}")
    print("")

    pscheduler_both = os.path.join(rootdir, p[2])
    filenames = traverse2(pscheduler_both)

    data_seg_sum_cubic, data_seg_sum_bbr2 = [], []
    tput_cubic_p16, tput_bbr2_p16 = [], []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            data = [json.loads(line) for line in open(f, 'r')]
            
            throughput_cubic, throughput_bbr2, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in zip(range(len(data)),data):
                try:
                    if "interval" in d.keys() and j==0:
                        start_time = d['interval']['time'] # Start time of the test
                    if "streams" in d.keys():
                        end_time = data[j-1]['interval']['time'] # End time of the test
                        mss = data[j-1]['interval']['mss'] # maximum segment size
                        
                        cubic_data_seg_list, bbr2_data_seg_list = [], []
                        for j in range(len(d['streams'])):
                            if "cubic" in d['streams'][j]['cc']:
                                cubic_data_seg_list.append(d['streams'][j]['data_segs'])
                            elif "bbr2" in d['streams'][j]['cc']:
                                bbr2_data_seg_list.append(d['streams'][j]['data_segs'])
                        
                        data_seg_sum_cubic.append( sum(cubic_data_seg_list) )
                        throughput_cubic = (sum(cubic_data_seg_list)*mss*8)/(end_time-start_time)/1e9
                        tput_cubic_p16.append( throughput_cubic )

                        data_seg_sum_bbr2.append( sum(bbr2_data_seg_list) )
                        throughput_bbr2 = (sum(bbr2_data_seg_list)*mss*8)/(end_time-start_time)/1e9
                        tput_bbr2_p16.append( throughput_bbr2 )

                except Exception as e:
                    print(e)

        except Exception as e:
            print(e)

    print("Throughput (P16)")
    print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p16):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p16):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p16)/np.mean(tput_bbr2_p16)):.5f}  |  Variance: {np.var(tput_bbr2_p16):.5f}")
    print(f"CUBIC - Mean: {np.mean(tput_cubic_p16):.5f}  |  Std. Dev.: {np.std(tput_cubic_p16):.5f}  |  Coef. of Variance: {(np.std(tput_cubic_p16)/np.mean(tput_cubic_p16)):.5f}  |  Variance: {np.var(tput_cubic_p16):.5f}")

    print("Data Segment (P16)")
    print(f"BBRv2 - Mean: {np.mean(data_seg_sum_bbr2):.5f}  |  Std. Dev.: {np.std(data_seg_sum_bbr2):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_bbr2)/np.mean(data_seg_sum_bbr2)):.5f}  |  Variance: {np.var(data_seg_sum_bbr2):.5f}")
    print(f"CUBIC - Mean: {np.mean(data_seg_sum_cubic):.5f}  |  Std. Dev.: {np.std(data_seg_sum_cubic):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_cubic)/np.mean(data_seg_sum_cubic)):.5f}  |  Variance: {np.var(data_seg_sum_cubic):.5f}")
    print(50*"=")

--- ('2021-06-09:14:35/pscheduler_bbr2_p1', '2021-06-09:14:35/pscheduler_cubic_p1', '2021-06-09:14:35/pscheduler_both_p16') ---
total files: 60
Throughput (P1)
BBRv2 - Mean: 8.04496  |  Std. Dev.: 1.26181  |  Coef. of Variance: 0.15685  |  Variance: 1.59217
Data Segment (P1)
BBRv2 - Mean: 6787779.80000  |  Std. Dev.: 1042608.80432  |  Coef. of Variance: 0.15360  |  Variance: 1087033118851.19336

total files: 60
Throughput (P1)
CUBIC - Mean: 8.78308  |  Std. Dev.: 1.42227  |  Coef. of Variance: 0.16193  |  Variance: 2.02285
Data Segment (P1)
CUBIC - Mean: 7410541.70000  |  Std. Dev.: 1175787.44599  |  Coef. of Variance: 0.15866  |  Variance: 1382476118144.94312

total files: 60
Throughput (P16)
BBRv2 - Mean: 3.62083  |  Std. Dev.: 0.65942  |  Coef. of Variance: 0.18212  |  Variance: 0.43484
CUBIC - Mean: 5.80462  |  Std. Dev.: 0.92845  |  Coef. of Variance: 0.15995  |  Variance: 0.86202
Data Segment (P16)
BBRv2 - Mean: 3186495.28333  |  Std. Dev.: 612129.28303  |  Coef. of Variance: 0.1

## Non-ESnet Hosts

In [18]:
import os
import json
import numpy as np
from pathlib import Path

rootdir = "/Users/eashan22/Desktop/Internship 2021/bbrv2/Brian's Project/bost-dtn"
# rootdir = "/home/eadhikarla/Desktop/Brian's Project/bost-dtn"

def traverse3(path):
    fileList = []
    c = 0
    for root, directories, files in os.walk(path):
        for file in files:
            if file.endswith("json") and file.find("es.net")==-1: # non-esnet host
                fileList.append(os.path.join(root,file))
                c+=1
    print(f"total files: {c}")
    return fileList

paths = [("2021-06-09:14:35/pscheduler_bbr2_p1", "2021-06-09:14:35/pscheduler_cubic_p1", "2021-06-09:14:35/pscheduler_both_p16"),
         ("2021-06-15:02:12/pscheduler_bbr2_p1", "2021-06-15:02:12/pscheduler_cubic_p1", "2021-06-15:02:12/pscheduler_both_p16"),
         ("2021-06-16:02:12/pscheduler_bbr2_p1", "2021-06-16:02:12/pscheduler_cubic_p1", "2021-06-16:02:12/pscheduler_both_p16")
        ]

for p in paths:
    print(f"--- {p} ---")
    pscheduler_bbr2  = os.path.join(rootdir, p[0])
    filenames = traverse3(pscheduler_bbr2)

    data_seg = []
    tput_bbr2_p1 = []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            data = [json.loads(line) for line in open(f, 'r')]
            
            bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in enumerate(data):
                if "interval" in d.keys() and j==0:
                    start_time = data[j]['interval']['time'] # Start time of the test
                elif key2 in data[j].keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size

                    bbr2_data_seg = data[j][key2]
                    data_seg.append( bbr2_data_seg )

                    throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)/1e9
                    tput_bbr2_p1.append( throughput )
        
        except Exception as e:
            print(e)

    print("Throughput (P1)")
    print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p1):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p1):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p1)/np.mean(tput_bbr2_p1)):.5f}  |  Variance: {np.var(tput_bbr2_p1):.5f}")
    print("Data Segment (P1)")
    print(f"BBRv2 - Mean: {np.mean(data_seg):.5f}  |  Std. Dev.: {np.std(data_seg):.5f}  |  Coef. of Variance: {(np.std(data_seg)/np.mean(data_seg)):.5f}  |  Variance: {np.var(data_seg):.5f}")
    print("")

    pscheduler_cubic = os.path.join(rootdir, p[1])
    filenames = traverse3(pscheduler_cubic)

    data_seg = []
    tput_p1_cubic = []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            # The MongoDB JSON dump has one object per line, so this works for me.
            data = [json.loads(line) for line in open(f, 'r')]
            
            cubic_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in enumerate(data):
                if "interval" in d.keys() and j==0:
                    start_time = data[j]['interval']['time'] # Start time of the test
                elif key1 in data[j].keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size

                    cubic_data_seg = data[j][key1]
                    data_seg.append( cubic_data_seg )

                    throughput = (cubic_data_seg*mss*8)/(end_time-start_time)/1e9
                    tput_p1_cubic.append( throughput )
        
        except Exception as e:
            print(e)

    print("Throughput (P1)")
    print(f"CUBIC - Mean: {np.mean(tput_p1_cubic):.5f}  |  Std. Dev.: {np.std(tput_p1_cubic):.5f}  |  Coef. of Variance: {(np.std(tput_p1_cubic)/np.mean(tput_p1_cubic)):.5f}  |  Variance: {np.var(tput_p1_cubic):.5f}")
    print("Data Segment (P1)")
    print(f"CUBIC - Mean: {np.mean(data_seg):.5f}  |  Std. Dev.: {np.std(data_seg):.5f}  |  Coef. of Variance: {(np.std(data_seg)/np.mean(data_seg)):.5f}  |  Variance: {np.var(data_seg):.5f}")
    print("")

    pscheduler_both = os.path.join(rootdir, p[2])
    filenames = traverse3(pscheduler_both)

    data_seg_sum_cubic, data_seg_sum_bbr2 = [], []
    tput_cubic_p16, tput_bbr2_p16 = [], []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            data = [json.loads(line) for line in open(f, 'r')]
            
            throughput_cubic, throughput_bbr2, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in zip(range(len(data)),data):
                try:
                    if "interval" in d.keys() and j==0:
                        start_time = d['interval']['time'] # Start time of the test
                    if "streams" in d.keys():
                        end_time = data[j-1]['interval']['time'] # End time of the test
                        mss = data[j-1]['interval']['mss'] # maximum segment size
                        
                        cubic_data_seg_list, bbr2_data_seg_list = [], []
                        for j in range(len(d['streams'])):
                            if "cubic" in d['streams'][j]['cc']:
                                cubic_data_seg_list.append(d['streams'][j]['data_segs'])
                            elif "bbr2" in d['streams'][j]['cc']:
                                bbr2_data_seg_list.append(d['streams'][j]['data_segs'])
                        
                        data_seg_sum_cubic.append( sum(cubic_data_seg_list) )
                        throughput_cubic = (sum(cubic_data_seg_list)*mss*8)/(end_time-start_time)/1e9
                        tput_cubic_p16.append( throughput_cubic )

                        data_seg_sum_bbr2.append( sum(bbr2_data_seg_list) )
                        throughput_bbr2 = (sum(bbr2_data_seg_list)*mss*8)/(end_time-start_time)/1e9
                        tput_bbr2_p16.append( throughput_bbr2 )

                except Exception as e:
                    print(e)

        except Exception as e:
            print(e)

    print("Throughput (P16)")
    print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p16):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p16):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p16)/np.mean(tput_bbr2_p16)):.5f}  |  Variance: {np.var(tput_bbr2_p16):.5f}")
    print(f"CUBIC - Mean: {np.mean(tput_cubic_p16):.5f}  |  Std. Dev.: {np.std(tput_cubic_p16):.5f}  |  Coef. of Variance: {(np.std(tput_cubic_p16)/np.mean(tput_cubic_p16)):.5f}  |  Variance: {np.var(tput_cubic_p16):.5f}")

    print("Data Segment (P16)")
    print(f"BBRv2 - Mean: {np.mean(data_seg_sum_bbr2):.5f}  |  Std. Dev.: {np.std(data_seg_sum_bbr2):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_bbr2)/np.mean(data_seg_sum_bbr2)):.5f}  |  Variance: {np.var(data_seg_sum_bbr2):.5f}")
    print(f"CUBIC - Mean: {np.mean(data_seg_sum_cubic):.5f}  |  Std. Dev.: {np.std(data_seg_sum_cubic):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_cubic)/np.mean(data_seg_sum_cubic)):.5f}  |  Variance: {np.var(data_seg_sum_cubic):.5f}")
    print(50*"=")

--- ('2021-06-09:14:35/pscheduler_bbr2_p1', '2021-06-09:14:35/pscheduler_cubic_p1', '2021-06-09:14:35/pscheduler_both_p16') ---
total files: 45
Throughput (P1)
BBRv2 - Mean: 3.31125  |  Std. Dev.: 1.18396  |  Coef. of Variance: 0.35756  |  Variance: 1.40175
Data Segment (P1)
BBRv2 - Mean: 15668815.11111  |  Std. Dev.: 8830355.74326  |  Coef. of Variance: 0.56356  |  Variance: 77975182552479.25000

total files: 45
Throughput (P1)
CUBIC - Mean: 3.50997  |  Std. Dev.: 1.28378  |  Coef. of Variance: 0.36575  |  Variance: 1.64809
Data Segment (P1)
CUBIC - Mean: 16442362.97778  |  Std. Dev.: 9547810.75456  |  Coef. of Variance: 0.58068  |  Variance: 91160690204895.26562

total files: 49
Throughput (P16)
BBRv2 - Mean: 3.17675  |  Std. Dev.: 1.76380  |  Coef. of Variance: 0.55522  |  Variance: 3.11100
CUBIC - Mean: 2.47428  |  Std. Dev.: 1.33931  |  Coef. of Variance: 0.54129  |  Variance: 1.79375
Data Segment (P16)
BBRv2 - Mean: 32046009.20408  |  Std. Dev.: 31874435.08762  |  Coef. of Varian

In [25]:
import os
import json
import numpy as np
import warnings
from pathlib import Path
import csv

warnings.filterwarnings("ignore")
rootdir = "/Users/eashan22/Desktop/Internship 2021/bbrv2/Brian's Project/"

def _filereader(path, verbose=False):
    with open(path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        tempList = []
        # This skips the first row of the CSV file.
        next(csv_reader)

        for enum, row in enumerate(csv_reader):
            if verbose:
                print(f'{enum}.\t{row[0]}, {row[1]}, {row[2]}')
            tempList.append( (row[0],row[1],row[2]) )
            line_count += 1
        print(f'Processed {line_count} lines.')
    return tempList

rtt_data = _filereader(os.path.join(rootdir,'test-hosts.bost-dtn.csv'),False)

0.	bost-pt1.es.net, 1.0ms, 10Gbps
1.	newy-pt1.es.net, 5.0ms, 10Gbps
2.	eqx-ash-pt1.es.net, 10.0ms, 10Gbps
3.	star-pt1.es.net, 20.0ms, 10Gbps
4.	kans-pt1.es.net, 31.0ms, 10Gbps
5.	denv-pt1.es.net, 41.0ms, 10Gbps
6.	albq-pt1.es.net, 49.0ms, 10Gbps
7.	snla-pt1.es.net, 50.0ms, 1Gbps
8.	sacr-pt1.es.net, 61.0ms, 10Gbps
9.	cern-773-pt1.es.net, 87.0ms, 10Gbps
10.	aofa-pt1.es.net, 5.0ms, 10Gbps
11.	pppl-pt1.es.net, 6.0ms, 10Gbps
12.	wash-pt1.es.net, 9.0ms, 10Gbps
13.	bois-pt1.es.net, 57.0ms, 10Gbps
14.	psl01.pic.es, 119.0ms, 1Gbps
15.	psb.hpc.utfsm.cl, 170.0ms, 1Gbps
16.	nsw-brwy-ps1.aarnet.net.au, 230.0ms,  10Gbps
17.	perfsonar-bw.sprace.org.br, 140.0ms,  10Gbps
18.	perfsonar-latency.grid.surfsara.nl, 73.0ms, 10Gbps
19.	btw-bw.t1.grid.kiae.ru, 109.0ms, 1Gbps
20.	fiona.sce.pennren.net, 13.0ms, 10Gbps
21.	perfsonar.oshean.org, 9.0ms, 10Gbps
22.	tau.ijs.si, 102.0ms, 10Gbps
23.	lcgps02.gridpp.rl.ac.uk, 75.0ms, 10Gbps
24.	perfsonar.na.infn.it, 108.0ms,  10Gbps
25.	perfsonar2.icepp.jp, 173.0ms, 10Gb

In [35]:
def traverse_less_than_30(path):
    fileList = []
    c = 0
    for root, directories, files in os.walk(path):
        for file in files:
            if file.startswith("ss") and file.endswith("json"):
                for enum, rtt_file in enumerate(rtt_data):
                    if file.find(rtt_file[0])!=-1 and float(rtt_file[1].split("ms")[0])<30:
                        print(file, enum)
                        fileList.append(os.path.join(root,file))
                        c+=1
    print(f"Total files: {c}")
    return fileList

x = traverse_less_than_30(os.path.join(rootdir,"bost-dtn-10G/2021-07-30:19:21/pscheduler_both_p16"))

ss:eqx-ash-pt1.es.net:4.json 2
ss:newy-pt1.es.net:2.json 1
ss:pppl-pt1.es.net:3.json 11
ss:aofa-pt1.es.net:4.json 10
ss:aofa-pt1.es.net:5.json 10
ss:bost-pt1.es.net:1.json 0
ss:pppl-pt1.es.net:2.json 11
ss:newy-pt1.es.net:3.json 1
ss:eqx-ash-pt1.es.net:5.json 2
ss:newy-pt1.es.net:4.json 1
ss:pppl-pt1.es.net:5.json 11
ss:aofa-pt1.es.net:2.json 10
ss:eqx-ash-pt1.es.net:2.json 2
ss:eqx-ash-pt1.es.net:3.json 2
ss:aofa-pt1.es.net:3.json 10
ss:star-pt1.es.net:1.json 3
ss:wash-pt1.es.net:1.json 12
ss:pppl-pt1.es.net:4.json 11
ss:newy-pt1.es.net:5.json 1
ss:star-pt1.es.net:2.json 3
ss:bost-pt1.es.net:4.json 0
ss:wash-pt1.es.net:2.json 12
ss:eqx-ash-pt1.es.net:1.json 2
ss:wash-pt1.es.net:3.json 12
ss:bost-pt1.es.net:5.json 0
ss:star-pt1.es.net:3.json 3
ss:aofa-pt1.es.net:1.json 10
ss:star-pt1.es.net:4.json 3
ss:bost-pt1.es.net:2.json 0
ss:wash-pt1.es.net:4.json 12
ss:pppl-pt1.es.net:1.json 11
ss:newy-pt1.es.net:1.json 1
ss:wash-pt1.es.net:5.json 12
ss:bost-pt1.es.net:3.json 0
ss:star-pt1.es.net